
# 🕹️ Setup Quest v3 — **Auto‑Install Miniconda + Ollama** · Kernel‑Safe o Entorno Aislado (*Sports Crew*)

Esta versión **puede instalar Miniconda y Ollama automáticamente** si detecta que no están presentes (macOS / Windows / Linux),
y mantiene dos rutas de trabajo:
- **Ruta A**: instalar en el **kernel actual** (rápido, sin entorno nuevo).
- **Ruta B**: crear un **entorno aislado** (Conda/Mamba o venv) y registrar kernel.

> ⚠️ Algunas instalaciones pueden requerir **permisos** o aceptar términos. Revisa los mensajes y confirma si se te solicita.



## ⛳ Pre‑Nivel −2 — **Miniconda** (instalación automática si falta)

**Documentación oficial:**
- Miniconda: https://docs.conda.io/en/latest/miniconda.html

### ¿Qué hará esta celda?
- Detectará tu sistema (macOS / Windows / Linux).
- Intentará instalar **Miniconda**:
  - **macOS**: primero con Homebrew (`brew install --cask miniconda`), si no hay brew intenta **instalador .sh** (arm64/x86_64).
  - **Windows**: intenta `winget install Anaconda.Miniconda3` (silencioso si es posible) y como alternativa `choco install miniconda3` (si tienes Chocolatey).
  - **Linux**: usa el instalador oficial `.sh` en modo batch.

> Al terminar, te sugerirá **inicializar conda** e iniciar una nueva sesión de shell si hace falta.


In [ ]:

import os, sys, platform, shutil, subprocess, pathlib

def run(cmd, shell=False):
    print("→", cmd if isinstance(cmd, str) else " ".join(cmd))
    try:
        if shell:
            return subprocess.check_call(cmd, shell=True)
        return subprocess.check_call(cmd)
    except subprocess.CalledProcessError as e:
        print("   ✖︎ Falló con código", e.returncode)
        return e.returncode

def exists(cmd):
    return shutil.which(cmd) is not None

def download(url, dest):
    import urllib.request
    print(f"Descargando: {url}")
    urllib.request.urlretrieve(url, dest)
    print("Guardado en:", dest)

system = platform.system().lower()
arch = platform.machine().lower()
print("Sistema:", system, "| Arquitectura:", arch)

conda_ok = exists("conda")
mamba_ok = exists("mamba")
print("¿Conda en PATH?:", "✅" if conda_ok else "❌", "| ¿Mamba?:", "✅" if mamba_ok else "❌")

if conda_ok or mamba_ok:
    print("✅ Ya tienes conda/mamba. Puedes saltar este paso.")
else:
    print("⌛ Intentando instalar Miniconda...")
    if system == "darwin":  # macOS
        if exists("brew"):
            rc = run(["brew", "install", "--cask", "miniconda"])
            if rc == 0:
                print("✅ Miniconda instalada con Homebrew.")
            else:
                print("⚠️ Brew falló, probamos instalador .sh")
        else:
            print("ℹ️ Homebrew no está disponible, usaré el instalador .sh")

        # Intento con instalador .sh (arm64 vs x86_64)
        if not exists("conda"):
            base = "https://repo.anaconda.com/miniconda"
            fname = "Miniconda3-latest-MacOSX-arm64.sh" if "arm" in arch or "aarch" in arch else "Miniconda3-latest-MacOSX-x86_64.sh"
            url = f"{base}/{fname}"
            installer = str(pathlib.Path.cwd() / fname)
            try:
                download(url, installer)
                run(["bash", installer, "-b", "-p", str(pathlib.Path.home() / "miniconda3")])
                print("✅ Miniconda instalada en ~/miniconda3")
                print("👉 Inicializa conda para tu shell (zsh por defecto):")
                print("   ~/miniconda3/bin/conda init zsh   &&  exec $SHELL -l")
            except Exception as e:
                print("✖︎ Error descargando/instalando Miniconda:", e)

    elif system == "windows":
        # Winget primero
        if exists("winget"):
            # flags para no bloquear interacción si es posible
            rc = run(["winget", "install", "--id", "Anaconda.Miniconda3", "-e", "--accept-source-agreements", "--accept-package-agreements"])
            if rc == 0:
                print("✅ Miniconda instalada con winget.")
            else:
                print("⚠️ Winget falló, intento con Chocolatey si existe...")
        # Chocolatey fallback
        if not exists("conda") and exists("choco"):
            rc = run(["choco", "install", "miniconda3", "-y"])
            if rc == 0:
                print("✅ Miniconda instalada con Chocolatey.")
        if not exists("conda"):
            print("ℹ️ Si no funcionó, descarga manualmente el instalador desde: https://docs.conda.io/en/latest/miniconda.html")

    else:  # Linux
        base = "https://repo.anaconda.com/miniconda"
        fname = "Miniconda3-latest-Linux-aarch64.sh" if "arm" in arch or "aarch" in arch else "Miniconda3-latest-Linux-x86_64.sh"
        url = f"{base}/{fname}"
        installer = str(pathlib.Path.cwd() / fname)
        try:
            download(url, installer)
            run(["bash", installer, "-b", "-p", str(pathlib.Path.home() / "miniconda3")])
            print("✅ Miniconda instalada en ~/miniconda3")
            print("👉 Inicializa conda para tu shell (bash/zsh):")
            print("   ~/miniconda3/bin/conda init && exec $SHELL -l")
        except Exception as e:
            print("✖︎ Error descargando/instalando Miniconda:", e)



## ⛳ Pre‑Nivel −1 — **Ollama** (instalación automática si falta)

**Documentación:**
- Sitio oficial: https://ollama.com/
- GitHub: https://github.com/ollama/ollama
- API: https://github.com/ollama/ollama/blob/main/docs/api.md

### ¿Qué hará esta celda?
- Detectará tu sistema (macOS / Windows / Linux).
- Intentará instalar **Ollama**:
  - **macOS**: `brew install --cask ollama` si hay Homebrew.
  - **Windows**: `winget install Ollama.Ollama` (o descarga desde la web).
  - **Linux**: script oficial `curl -fsSL https://ollama.com/install.sh | sh`.

> Tras instalar, asegúrate de **iniciar el servicio** (`ollama serve`) y descarga `mistral` con `ollama pull mistral`.


In [ ]:

import os, sys, platform, shutil, subprocess

def run(cmd, shell=False):
    print("→", cmd if isinstance(cmd, str) else " ".join(cmd))
    try:
        if shell:
            return subprocess.check_call(cmd, shell=True)
        return subprocess.check_call(cmd)
    except subprocess.CalledProcessError as e:
        print("   ✖︎ Falló con código", e.returncode)
        return e.returncode

def exists(cmd):
    return shutil.which(cmd) is not None

system = platform.system().lower()
print("Sistema:", system)
ollama_ok = exists("ollama")
print("¿Ollama en PATH?:", "✅" if ollama_ok else "❌")

if ollama_ok:
    print("✅ Ya tienes Ollama. Puedes saltar este paso.")
else:
    print("⌛ Intentando instalar Ollama...")
    if system == "darwin":  # macOS
        if exists("brew"):
            run(["brew", "install", "--cask", "ollama"])
        else:
            print("ℹ️ Homebrew no está disponible. Instala Ollama manualmente desde https://ollama.com/download")
    elif system == "windows":
        if exists("winget"):
            run(["winget", "install", "Ollama.Ollama", "-e", "--accept-source-agreements", "--accept-package-agreements"])
        else:
            print("ℹ️ Instala Ollama manualmente desde https://ollama.com/download (o usa winget si está disponible).")
    else:  # Linux
        # Script oficial
        rc = run("curl -fsSL https://ollama.com/install.sh | sh", shell=True)
        if rc != 0:
            print("ℹ️ Si falló, revisa la guía oficial: https://github.com/ollama/ollama")

print("👉 Luego ejecuta en terminal:")
print("   ollama serve   # iniciar servicio")
print("   ollama pull mistral  # descargar modelo")


## 🧩 Nivel 0 — Comprobación del sistema

In [ ]:

import sys, platform, subprocess
print("Python:", sys.version)
print("Ejecutable:", sys.executable)
print("Sistema:", platform.platform())

out = subprocess.check_output([sys.executable, "-m", "pip", "--version"], text=True)
print(out.strip())

ok = sys.version_info >= (3, 9)
print("\nRequisito Python>=3.9:", "✅" if ok else "❌  (Se recomienda 3.10/3.11)")



---
## 🧪 Crear **entorno aislado** (Conda/Mamba o venv) + registrar kernel

Instalar dependencias por proyecto.


### 1) Detectar gestor disponible

In [ ]:

import shutil
has_mamba = shutil.which("mamba") is not None
has_conda = shutil.which("conda") is not None
print("Mamba:", "✅" if has_mamba else "—")
print("Conda:", "✅" if has_conda else "—")
print("Si ninguno está disponible, usaremos 'venv'. (Puedes volver a Pre‑Nivel −2 para instalar Miniconda)")



### 2) Crear entorno `unex25` (Conda/Mamba) o `.venv_unex25` (venv)

**Docs:**  
- Conda envs: https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html  
- venv: https://docs.python.org/3/library/venv.html


In [ ]:

import sys, subprocess, pathlib, os, shutil

TARGET_ENV_NAME = "unex25"
TARGET_VENV_DIR = pathlib.Path.cwd() / ".venv_unex25"

def run(cmd):
    print("→", " ".join(cmd))
    subprocess.check_call(cmd)

has_mamba = shutil.which("mamba") is not None
has_conda = shutil.which("conda") is not None

if has_mamba or has_conda:
    conda = shutil.which("mamba") or shutil.which("conda")
    run([conda, "create", "-n", TARGET_ENV_NAME, "python=3.10", "-y"])
    print("✅ Entorno conda/mamba creado:", TARGET_ENV_NAME)
else:
    if not TARGET_VENV_DIR.exists():
        run([sys.executable, "-m", "venv", str(TARGET_VENV_DIR)])
        print("✅ Entorno venv creado:", TARGET_VENV_DIR)
    else:
        print("ℹ️ Entorno venv ya existía:", TARGET_VENV_DIR)

print("\n👉 #####Ahora activa el entorno#########")
print("\nAhora, en Jupyter: Kernel → Change kernel → 'Python (unex25)'.")



### 3) Instalar dependencias **dentro del entorno**
> Evitamos `%pip` aquí para asegurarnos de instalar en **ese** entorno.


In [ ]:

import sys, subprocess, pathlib, os, shutil

PKGS = [
    "crewai==0.177.0",
    "litellm==1.74.9",
    "langchain==0.3.27",
    "langchain-core==0.3.75",
    "langchain-community==0.3.29",
    "langchain-openai==0.3.32",
    "openai==1.106.1",
    "ollama==0.5.3",
    "tiktoken==0.11.0",
    "huggingface-hub==0.34.4",
    "rich==14.1.0",
    "httpx==0.28.1",
    "httpcore==1.0.9",
    "requests==2.32.5",
    "urllib3==2.5.0",
    "tenacity==9.1.2",
    "pydantic==2.11.7",
    "pydantic-core==2.33.2",
    "python-dotenv==1.1.1",
    "SQLAlchemy==2.0.43",
    "chromadb==1.0.20",
    "numpy==2.2.6",
    "pandas==2.3.2",
    "Jinja2==3.1.6",
    "duckduckgo_search==8.1.1",
    "orjson==3.11.3",
    "grpcio==1.74.0",
    "ddgs==9.5.5",
    "neo4j==5.28.2",
]

has_mamba = shutil.which("mamba") is not None
has_conda = shutil.which("conda") is not None

if has_mamba or has_conda:
    conda = shutil.which("mamba") or shutil.which("conda")
    cmd = [conda, "run", "-n", "unex25", sys.executable, "-m", "pip", "install", "-U"] + PKGS
    print("Instalando en conda/mamba env 'unex25'...")
    subprocess.check_call(cmd)
else:
    vpy = (pathlib.Path.cwd() / ".venv_unex25" / ("Scripts" if os.name=="nt" else "bin") / "python")
    if not vpy.exists():
        raise SystemExit("❌ No se encontró el venv .venv_unex25. Ejecuta el paso B2 primero.")
    cmd = [str(vpy), "-m", "pip", "install", "-U"] + PKGS
    print("Instalando en venv '.venv_unex25'...")
    subprocess.check_call(cmd)

print("✅ Dependencias instaladas en el entorno aislado.")


### 4) Registrar kernel **Python (unex25)** desde el entorno

In [ ]:

import sys, subprocess, pathlib, os, shutil

has_mamba = shutil.which("mamba") is not None
has_conda = shutil.which("conda") is not None

if has_mamba or has_conda:
    conda = shutil.which("mamba") or shutil.which("conda")
    cmd = [conda, "run", "-n", "unex25", sys.executable, "-m", "ipykernel", "install", "--user", "--name", "unex25", "--display-name", "Python (unex25)"]
    subprocess.check_call(cmd)
    print("✅ Kernel registrado: Python (unex25)")
else:
    vpy = (pathlib.Path.cwd() / ".venv_unex25" / ("Scripts" if os.name=="nt" else "bin") / "python")
    if not vpy.exists():
        raise SystemExit("❌ No se encontró el venv .venv_unex25. Ejecuta el paso B2 primero.")
    cmd = [str(vpy), "-m", "ipykernel", "install", "--user", "--name", "unex25", "--display-name", "Python (unex25)"]
    subprocess.check_call(cmd)
    print("✅ Kernel registrado: Python (unex25)")


---
## 🔎 Checks comunes (válidos para ambas rutas)

### ✅ Imports (en el kernel actual)

In [ ]:

import importlib
mods = ["crewai", "litellm", "langchain", "langchain_community", "langchain_openai", "duckduckgo_search", "requests", "dotenv", "ipykernel", "neo4j", "ddgs"]
for m in mods:
    try:
        importlib.import_module(m)
        print(f"{m:24} ✅")
    except Exception as e:
        print(f"{m:24} ❌ -> {e}")


### 📰 ESPN + 🔎 DuckDuckGo (LangChain)

In [ ]:

import requests
try:
    from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
    try:
        from langchain_community.tools import DuckDuckGoSearchRun
    except Exception:
        from langchain.tools import DuckDuckGoSearchRun
    ddg_ok = True
except Exception as e:
    ddg_ok = False
    print("DDG import falló:", e)

def get_sport_news_url(sport: str, league: str) -> str:
    base_url = "http://site.api.espn.com/apis/site/v2/sports"
    return f"{base_url}/{sport}/{league}/news"

def fetch_articles(url: str):
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        data = r.json()
        if isinstance(data, dict):
            if "articles" in data and isinstance(data["articles"], list):
                return data["articles"]
            if "feed" in data and isinstance(data["feed"], dict):
                entries = data["feed"].get("entries", [])
                return [{"headline": e.get("headline") or e.get("title", "")} for e in entries]
        return []
    except Exception as e:
        print("Aviso:", e)
        return []

sport, league = "soccer", "esp.1"
arts = fetch_articles(get_sport_news_url(sport, league))
print(f"Artículos ESPN ({sport}/{league}):", len(arts))

if ddg_ok:
    wrapper = DuckDuckGoSearchAPIWrapper(region="us-en", time="d")
    searcher = DuckDuckGoSearchRun(api_wrapper=wrapper)
    q = "LaLiga resultados"
    print("DDG consulta:", q)
    res = searcher.run(q)
    print("DDG respuesta (primeros 300 chars):\n", (res or "")[:300].replace("\n", " "))
    print("✅ DDG OK")
else:
    print("⚠️ Salto la prueba de DDG porque los imports fallaron arriba.")


### 🤖 Ollama — comprobación de servicio y modelo

In [ ]:

import shutil, requests
print("Binario 'ollama':", "✅" if shutil.which("ollama") else "❌ (no encontrado)")
try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    if r.ok:
        tags = [m.get("name") for m in r.json().get("models", []) if isinstance(m, dict)]
        print("Ollama API:", "✅ OK")
        print("Modelos locales:", tags)
        print("¿'mistral' presente?:", "✅" if any("mistral" in (t or "") for t in tags) else "❌")
    else:
        print("Ollama API respondió con error HTTP:", r.status_code)
except Exception as e:
    print("No se pudo conectar a Ollama en http://localhost:11434 — ¿está 'ollama serve' activo?")
    print("Detalle:", e)


### 🔐 OpenAI (opcional)

In [ ]:

import os
from getpass import getpass
try:
    from langchain_openai import ChatOpenAI
    key = os.getenv("OPENAI_API_KEY") or ""
    if not key:
        print("No hay OPENAI_API_KEY en el entorno (opcional).")
        try:
            key = getpass("Introduce tu OPENAI_API_KEY (no se mostrará): ")
            if key:
                os.environ["OPENAI_API_KEY"] = key
        except Exception:
            pass
    if os.getenv("OPENAI_API_KEY"):
        llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL","gpt-4o-mini"), temperature=0.0)
        resp = llm.invoke("Responde únicamente: ok")
        print("OpenAI:", "✅" if "ok" in (getattr(resp, "content", str(resp)) or "").lower() else "respuesta inesperada")
    else:
        print("⚠️ Saltando prueba de OpenAI (sin API key).")
except Exception as e:
    print("❌ Prueba de OpenAI falló:", e)


### 🧪 Mini smoke test: CrewAI + Ollama (JSON estricto)

In [ ]:

from crewai import Agent, Task, LLM
import json

try:
    llm_local = LLM(model="ollama/mistral", base_url="http://localhost:11434", temperature=0.0, api_key="ollama-local")
    agent = Agent(role="JSONBot", goal="Devolver SOLO JSON válido", backstory="JSON-first", llm=llm_local, verbose=True)
    prompt = "Devuelve únicamente este objeto JSON: {\"ok\": true} sin texto extra."
    task = Task(agent=agent, description=prompt, expected_output="JSON {\"ok\": true}")
    out = task.execute_sync()
    s = out if isinstance(out, str) else str(out)
    start, end = s.find("{"), s.rfind("}")
    block = s[start:end+1] if start!=-1 and end!=-1 else s
    obj = json.loads(block)
    print("Respuesta JSON:", obj)
    print("✅ CrewAI + Ollama OK")
except Exception as e:
    print("❌ Falla CrewAI + Ollama:", e)
    print("Pistas: ¿Ollama está sirviendo? ¿Descargaste el modelo con 'ollama pull mistral'?")
